# Automatic Number Plate Detection Algorithm Integration

# Source Codes

In [ ]:
#!pip install easyocr
#!pip install imutils

In [ ]:
# importing Libraries 
import cv2
from google.colab.patches import cv2_imshow
import easyocr
import imutils
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Load the pre-trained Haar cascade classifier for license plate detection
plate_cascade = cv2.CascadeClassifier('indian_number_plate.xml')

# Check if the cascade classifier is loaded successfully
if plate_cascade.empty():
    raise IOError('Unable to load the number plate cascade classifier XML file.')


# Number Plate Text Detection

In [ ]:
images = ['data14.png','data17.png','data11.png','data9.png']    # input images
 
for i in images:                                                 # applied for_loop to run one by one
  
    img = cv2.imread(i)                                            # pulling data in opencv2 format

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                   # scaling 3channel to 2 channel (gray scale or black white color image) 
   
  bfilter = cv2.bilateralFilter(gray, 11, 17, 17)                #Noise reduction need gray scale image

    edged = cv2.Canny(bfilter, 30, 200)                            #Edge detection or draw boundary line 

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # give x,y values of edges point

    contours = imutils.grab_contours(keypoints)                    #  Grab x y cordinates objects   
             
                                                                 #sorting reverse and geting top 10

    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]  

    location = None                                                # giving none value to location

    for contour in contours:                                       # applyong fo_loop for top 10 counters
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:                                         # collecting only 4 points counter and saving in location variable
        location = approx
        break

    mask = np.zeros(gray.shape, np.uint8)                          # saving empty container  in mask variable

    new_image = cv2.drawContours(mask, [location], 0,255, -1)      #applying mask to identify the number plate location
    new_image = cv2.bitwise_and(img, img, mask=mask)

    (x,y) = np.where(mask==255)                                   # storing min max weight and height in new variable
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))

    cropped_image = gray[x1:x2+1, y1:y2+1]                        # cropped image

    reader = easyocr.Reader(['en'])                               # collecting text from cropped image using easyocr library
    result = reader.readtext(cropped_image) 
    text = result[0][-2]                                          # collecting text value from array

                                                                # collecting dimension of number plate
    print("Number Plate Detected : ",text)                        # extracting text
    print(" ")            


# Replacing Number plate to Company logo

In [ ]:
plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=15, minSize=(50, 50))  
  
    for (x, y, w, h) in plates:
                                                                # Load the logo image
    logo = cv2.imread('logo.png')

                                                                # Resize the logo image to match the dimensions of the number plate region
    resized_logo = cv2.resize(logo, (w, h))

                                                                # Replace the number plate region with the logo
    img[y:y+h, x:x+w] = resized_logo


    cv2_imshow(img)                                             # extracting replaced number plate images by company logo             
